In [18]:
from playwright.async_api import async_playwright
import asyncio

async def scrape_room_details(url):
    room_details = {
        'url': url,
        'title': None,
        'price_per_night': None,
        'total_price': None,
        'rating': None,
        'num_reviews': None,
        'superhost': False,
        'amenities': [],
        'bedrooms': None,
        'bathrooms': None,
        'max_guests': None
    }

    async with async_playwright() as p:
        # You can choose chromium, firefox, or webkit
        browser = await p.firefox.launch(headless=True)
        page = await browser.new_page()
        await page.goto(url, timeout=60000)

        # Wait a bit to allow page elements to load
        await page.wait_for_timeout(5000)

        # Decline translation prompt if it appears
        try:
            button = await page.query_selector("button:has-text('Show original')")
            if button:
                await button.click()
                await page.wait_for_timeout(2000)
        except Exception as e:
            print("Translation prompt not handled:", e)

        # Scrape room title
        try:
            title_elem = await page.wait_for_selector("h1", timeout=10000)
            room_details['title'] = await title_elem.inner_text()
        except Exception as e:
            print("Title not found:", e)

        # Scrape price per night
        try:
            price_elem = await page.query_selector("span._tyxjp1")
            if price_elem:
                room_details['price_per_night'] = await price_elem.inner_text()
        except Exception as e:
            print("Price per night not found:", e)

        # Scrape total price
        try:
            total_elem = await page.query_selector("span._1k4xcdh")
            if total_elem:
                room_details['total_price'] = await total_elem.inner_text()
        except Exception as e:
            print("Total price not found:", e)

        # Scrape rating
        try:
            rating_elem = await page.query_selector("span._17p6nbba")
            if rating_elem:
                room_details['rating'] = await rating_elem.inner_text()
        except Exception as e:
            print("Rating not found:", e)

        # Scrape number of reviews
        try:
            reviews_elem = await page.query_selector("span._s65ijh7")
            if reviews_elem:
                room_details['num_reviews'] = await reviews_elem.inner_text()
        except Exception as e:
            print("Number of reviews not found:", e)

        # Check for Superhost status
        try:
            superhost_elem = await page.query_selector("div:has-text('Superhost')")
            if superhost_elem:
                room_details['superhost'] = True
        except Exception as e:
            print("Superhost element not found:", e)

        # Scrape amenities
        try:
            amenities_elems = await page.query_selector_all("div._1nlbjeu")
            room_details['amenities'] = [await elem.inner_text() for elem in amenities_elems]
        except Exception as e:
            print("Amenities not found:", e)

        # Scrape details like bedrooms, bathrooms, and max guests
        try:
            details_elem = await page.query_selector("div._tqmy57")
            if details_elem:
                details_text = await details_elem.inner_text()
                if 'bedroom' in details_text:
                    room_details['bedrooms'] = details_text.split(' bedroom')[0].split()[-1]
                if 'bathroom' in details_text:
                    room_details['bathrooms'] = details_text.split(' bathroom')[0].split()[-1]
                if 'guest' in details_text:
                    room_details['max_guests'] = details_text.split(' guest')[0].split()[-1]
        except Exception as e:
            print("Room details (bedrooms/bathrooms/guests) not found:", e)

        await browser.close()
    return room_details

# Example usage:
if __name__ == "__main__":
    url = "https://www.airbnb.com/rooms/1176360685707736484?search_mode=regular_search&adults=1&check_in=2025-04-15&check_out=2025-04-20&children=0&infants=0&pets=0"
    data = asyncio.run(scrape_room_details(url))
    print("\nRoom Details:")
    for key, value in data.items():
        print(f"{key}: {value}")


RuntimeError: asyncio.run() cannot be called from a running event loop

https://www.airbnb.com/s/Belo-Horizonte--MG/homes?flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2025-03-01&monthly_length=3&monthly_end_date=2025-06-01&adults=0&place_id=ChIJMyzPysqQpgARlznSOl55NVs&refinement_paths%5B%5D=%2Fhomes&location_bb=wZ415cIvdYvBoDuiwjBA1Q%3D%3D&acp_id=8949c462-3712-48e3-8bed-185932a32fe1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click&

https://www.airbnb.com/rooms/1176360685707736484?search_mode=regular_search&adults=1&check_in=2025-04-15&check_out=2025-04-20&children=0&infants=0&pets=0&source_impression_id=p3_1740166446_P3tNP51VZhf7b5cZ&previous_page_section_name=1000&federated_search_id=674aca58-3b10-4ba4-b016-a2110258ce86